In [1]:
from gremlin_python import statics # type: ignore
from gremlin_python.structure.graph import Graph # type: ignore
from gremlin_python.process.graph_traversal import __ # type: ignore
from gremlin_python.process.strategies import * # type: ignore
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection # type: ignore
from gremlin_python.driver.aiohttp.transport import AiohttpTransport # type: ignore
from gremlin_python.process.traversal import * # type: ignore
from gremlin_python.process.traversal import Cardinality # type: ignore
from gremlin_python.process.traversal import T # type: ignore

import pandas as pd

port = 8182
server = '{Neptuneのエンドポイント}'
    
endpoint = f'wss://{server}:{port}/gremlin'

graph = Graph()

connection = None

try:
    connection = DriverRemoteConnection(endpoint, 'g',
                                        transport_factory=lambda: AiohttpTransport(call_from_event_loop=True))

    g = graph.traversal().withRemote(connection)

    # 一旦すべてのノードを削除
    g.V().drop().iterate()

    # ノードを追加
    g.addV('compute').property(T.id, '1').property('name', 'EC2').iterate()
    g.addV('compute').property(T.id, '2').property('name', 'Lambda').property('aritecture', 'container').iterate()
    g.addV('compute').property(T.id, '3').property('name', 'ECS').property('aritecture', 'serverless').iterate()
    g.addV('networking & container delivery').property(T.id, '4').property('name', 'ELB').iterate()
    g.addV('application integretion').property(T.id, '5').property('name', 'API Gateway').iterate()
    g.addV('endpoint').property(T.id, '6').property('name', 'HTTP').iterate()
    g.addV('endpoint').property(T.id, '7').property('name', 'REST').iterate()
    g.addV('storage').property(T.id, '8').property('name', 'EBS').iterate()
    g.addV('storage').property(T.id, '9').property('name', 'EFS').property('aritecture', 'network storage').iterate()
    g.addV('instance type').property(T.id, '10').property('name', '一般').property('first character', 'm').iterate()
    g.addV('instance type').property(T.id, '11').property('name', 'コンピューティング最適化').property('first character', 'c').iterate()
    g.addV('instance type').property(T.id, '12').property('name', 'メモリ最適化').property('first character', 'r').iterate()

    # エッジを追加 (ELBとEC2)
    g.V('4').addE('connect').to(__.V('1')).iterate()
    # エッジを追加 (ELBとECS)
    g.V('4').addE('connect').to(__.V('3')).iterate()
    # エッジを追加 (ELB > API Gateway > Lambda)
    g.V('4').addE('connect').to(__.V('5')).iterate()  # ELB -> API Gateway
    g.V('5').addE('connect').to(__.V('2')).iterate()  # API Gateway -> Lambda
    # EC2とinstance typeを繋げる
    g.V('1').addE('instance type').to(__.V('10')).iterate()
    g.V('1').addE('instance type').to(__.V('11')).iterate()
    g.V('1').addE('instance type').to(__.V('12')).iterate()
    # EC2とstorageを繋げる
    g.V('1').addE('mount').to(__.V('8')).iterate()
    g.V('1').addE('mount').to(__.V('9')).iterate()

    # API Gatewayとendpointを繋げる
    g.V('5').addE('endpoint').to(__.V('6')).iterate()
    g.V('5').addE('endpoint').to(__.V('7')).iterate()

    # EC2とインスタンスタイプの情報を取得し、表形式で表示する
    ec2_instance_type_info = g.V().has('name', 'EC2') \
        .as_('ec2') \
        .outE('instance type').inV() \
        .project('compute_name', 'instance_type_name', 'first_character') \
        .by(__.select('ec2').values('name')) \
        .by(__.values('name')) \
        .by(__.values('first character')) \
        .toList()

    # 結果をDataFrameに変換
    df = pd.DataFrame(ec2_instance_type_info)

    # 結果を表示
    print(df)
    
finally:
    if connection is not None:
        connection.close()

ClientConnectorError: Cannot connect to host graphdb-develop-neptune-cluster.cluster-causxyo47g34.ap-northeast-1.neptune.amazonaws.com:8183 ssl:default [Connect call failed ('10.192.10.61', 8183)]